# Policy gradient

In [2]:
import numpy as np
import tensorflow as tf

In [2]:
import gym

# Будем обучать агента, используя сначала простое окружение CartPole-v0,
# а затем усложним задачу взяв Acrobot-v1

# Создаем окружение 
# Здесь ваш код
env = 

observation_shape = env.observation_space.shape
n_actions = env.action_space.n
gamma = 0.95

print("Observation Space", env.observation_space)
print("Action Space", env.action_space)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Observation Space Box(6,)
Action Space Discrete(3)


# Reinforce algorithm

In [3]:
# Задаем переменные, которые будут подаваться на вход нейронной сети

# Состояния
# Здесь ваш код
observations =

# Совершенные действия
# Здесь ваш код
actions =

# Вознаграждение
# Здесь ваш код
discounted_episode_rewards =

all_inputs = [observations, actions, discounted_episode_rewards]

## Define network

In [4]:
sess = tf.InteractiveSession()

# Задаем внутренние и выходной слои нейронной сети
# Здесь ваш код
nn1 = 
nn2 = 
nn3 =  

probs_out = tf.nn.softmax(nn3) 
# Выход последнего слоя преобразуется в стохастическую стратегию, 
# поэтому количество нейронов должно быть равно n_actions

In [5]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    
    # Считаем дисконтированное вознаграждение "G = r + gamma*r' + gamma^2*r'' + ..."
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    # Нормализуем данные
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    
    return discounted_episode_rewards

Определите функцию потерь (Crossentropy loss)

Градиент стратегии выглядит следующим образом $\nabla_{\theta}J_{\theta}\approx\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^T\nabla_{\theta}log\pi_{\theta}(a_{i,t}|s_{i,t})G $. Чтобы автоматически вычислить градиент, необходимо задать граф, который имеет градиент такого же вида. Для этого используется "псевдо" функция потерь: $\tilde{J}_{\theta}=\frac{1}{N}\sum_{i=1}^N\sum_{t=1}^Tlog\pi_{\theta}(a_{i,t}|s_{i,t})G $.
Подробнее в четвертой лекции http://rail.eecs.berkeley.edu/deeprlcourse-fa17/index.html#lecture-videos (6 сентября)

In [7]:
# Применяем кросс энтропию к последнему слою сети tf.nn.softmax_cross_entropy_with_logits_v2
# Здесь ваш код
neg_log_prob = 

# Умножаем на дисконтированное вознаграждение и берем среднее, чтобы получить искомую функцию потерь
# Здесь ваш код
loss =     

# Задаем метод оптимизации, который будем использовать (например adam с lr 0.01)
# Здесь ваш код
optimizer = 
sess.run(tf.global_variables_initializer())

Instructions for updating:
Please switch to tf.train.get_global_step


In [1]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]
max_episodes = 1000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for episode in range(max_episodes):
        
        episode_rewards_sum = 0
        state = env.reset()
        # Можно посмотреть как выглядит окружение, если мы не используема google colab
#         env.render()
           
        while True:
            
            # Получаем распределение вероятностей действий в соответствии со стохастической стратегией агента
            # Здесь ваш код
            probs = 
            
            # Выбираем действие (согласно распределению)
            # Здесь ваш код
            action =  
            
            # Применяем действие в среде 
            # Здесь ваш код
            new_state, reward, done, info = 

            episode_states.append(state)
            action_ = np.zeros(n_actions)
            action_[action] = 1
            
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            if done:
                episode_rewards_sum = np.sum(episode_rewards)
                
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                
                mean_reward = np.divide(total_rewards, episode+1)
                
                
                maximumRewardRecorded = np.amax(allRewards)
                if episode % 50 == 0:
                    print("==========================================")
                    print("Episode: ", episode)
                    print("Reward: ", episode_rewards_sum)
                    print("Mean Reward", mean_reward)
                    print("Max reward so far: ", maximumRewardRecorded)
                
                # Считаем дисконтированное вознаграждение
                # Здесь ваш код
                discounted_rewards = 
                                
                # Считаем значение функции потерь и настраиваем веса сети при помощи оптимизатора
                # Здесь ваш код
                loss_, _ = 
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            state = new_state

SyntaxError: invalid syntax (<ipython-input-1-87e54dcb44a8>, line 22)